# Movie Recommendation System

In [2]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2020-08-20 21:12:49--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip      0%[                    ] 624.00K   174KB/s    eta 14m 55s^C
unziping ...
Archive:  moviedataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of moviedataset.zip or
        moviedataset.zip.zip, and cannot find m

In [2]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read the **movies.csv** file into their Dataframes:


In [9]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Let's also remove the year from the **title** column by using pandas' replace function and store in a new **year** column.


In [10]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also split the values in the **Genres** column into a **list of Genres** to simplify future use. This can be achieved by applying Python's split string function on the correct column.


In [11]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [12]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.


In [13]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let us take input from user : 

Notice: To add more movies, simply increase the amount of elements in the **userInput**. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [14]:
userInput = [
            {'title':'Don 2', 'rating':4.7321},
            {'title':'The Island', 'rating':4.5},
            {'title':'Conjuring, The', 'rating':3.1234},
            {'title':'7500', 'rating':4.8},
            {'title':'Bhool Bhulaiyaa', 'rating':5}
         ] 

inputMovies = pd.DataFrame(userInput)
inputMovies

,rating,title
0,4.7321,Don 2
1,4.5000,The Island
2,3.1234,"Conjuring, The"
3,4.8000,7500
4,5.0000,Bhool Bhulaiyaa


#### Add movieId to input user

With the input complete, let's extract the input movie's ID's from the movies dataframe and add them into it.

In [15]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,63515,The Island,4.5000
1,103688,"Conjuring, The",3.1234
2,109093,Don 2,4.7321
3,111856,7500,4.8000
4,134332,Bhool Bhulaiyaa,5.0000


#### The users who have seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.


In [16]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
14604,160,134332,3.5
29548,343,63515,3.5
42237,474,103688,4.5
48714,551,103688,3.0
64143,707,103688,3.5


We now group up the rows by user ID.


In [17]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.


In [19]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

[(38949,          userId  movieId  rating
  3600016   38949   103688     4.0
  3600052   38949   109093     3.0
  3600174   38949   134332     4.5), (71115,          userId  movieId  rating
  6622753   71115   103688     3.5
  6622802   71115   109093     2.5
  6622878   71115   134332     3.0), (5040,         userId  movieId  rating
  470353    5040   103688     3.5
  470385    5040   111856     1.0)]

#### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the **Pearson Correlation Coefficient**. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 \* Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.


We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.


In [118]:
userSubsetGroup = userSubsetGroup[0:100]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient

In [149]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

pearsonCorrelationDict.items()

[(10759, 0),
 (4105, 0),
 (239629, 1.0000000000000038),
 (1040, 0),
 (221732, -0.999999999999998),
 (38949, -0.0577365386056021),
 (551, 0),
 (177719, 0),
 (12640, 0),
 (11333, 0),
 (7755, 0),
 (6157, 0),
 (13399, 0),
 (11357, 0),
 (2655, 0),
 (1125, 0),
 (9319, 0),
 (8303, 0),
 (4208, 0),
 (7796, 0),
 (106102, -0.9999999999998969),
 (2921, 0),
 (2169, 0),
 (11898, 0),
 (67204, 0.9999999999999986),
 (7814, 0),
 (132751, -0.999999999999998),
 (9552, 0),
 (13972, 0),
 (9880, 0),
 (160, 0),
 (8360, 0),
 (4280, 0),
 (8379, 0),
 (707, 0),
 (146116, -0.999999999999998),
 (221475, 0),
 (5336, 0),
 (5852, 0),
 (215777, -0.9999999999999986),
 (138980, 0),
 (10473, 0),
 (14588, 1.0000000000000089),
 (71945, -0.999999999999998),
 (58641, -0.9999999999999959),
 (226070, -1.0000000000000002),
 (13444, 0),
 (83234, 0.9999999999999986),
 (207139, -1.0000000000000002),
 (2854, 0),
 (13097, 0),
 (12076, 0),
 (7982, -0.9999999999999986),
 (815, 0),
 (1848, 0),
 (1339, 0),
 (9030, 0),
 (12617, 0),
 (6199

In [150]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.0,10759
1,0.0,4105
2,1.0,239629
3,0.0,1040
4,-1.0,221732


#### The top x similar users to input user

Now let's get the top 50 users that are most similar to the input.

In [151]:
#print(topUsers.shape)
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()
#print(topUsers.shape)

,similarityIndex,userId
42,1.0,14588
66,1.0,15209
2,1.0,239629
72,1.0,161178
24,1.0,67204


Now, let's start recommending movies to the input user.

#### Rating of selected users to all movies

We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our **pearsonDF** from the ratings dataframe and then store their correlation in a new column called \_similarityIndex". This is achieved below by merging of these two tables.

In [152]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head(100)

,similarityIndex,userId,movieId,rating
0,1.0,14588,1,4.0
1,1.0,14588,2,3.5
2,1.0,14588,3,3.0
3,1.0,14588,5,2.0
4,1.0,14588,6,4.0
5,1.0,14588,7,3.0
6,1.0,14588,10,3.0
7,1.0,14588,15,2.0
8,1.0,14588,16,3.5
9,1.0,14588,18,4.0


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [153]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,14588,1,4.0,4.0
1,1.0,14588,2,3.5,3.5
2,1.0,14588,3,3.0,3.0
3,1.0,14588,5,2.0,2.0
4,1.0,14588,6,4.0,4.0


In [154]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,6.0,22.5
2,6.0,20.5
3,1.0,3.0
5,3.0,9.5
6,5.0,17.5


In [155]:
#Creates an empty dataframe
recommendation_df_collab = pd.DataFrame()
#Now we take the weighted average
recommendation_df_collab['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df_collab['movieId'] = tempTopUsersRating.index
recommendation_df_collab.head()

,weighted average recommendation score,movieId
movieId,,
1,3.750000,1
2,3.416667,2
3,3.000000,3
5,3.166667,5
6,3.500000,6


Now let's sort it and see the top 20 movies that the algorithm recommended!

In [157]:
recommendation_df_collab = recommendation_df_collab.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df_collab['weighted average recommendation score']
recommendation_df_collab.head(100)

,weighted average recommendation score,movieId
movieId,,
118392,5.000000,118392
36086,5.000000,36086
2575,5.000000,2575
4502,5.000000,4502
102070,5.000000,102070
4501,5.000000,4501
5304,5.000000,5304
2488,5.000000,2488
33564,5.000000,33564


In [192]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df_collab.head(20)['movieId'].tolist())]

,movieId,title,genres,year
1965,2048,"Great Mouse Detective, The","[Action, Animation, Children, Crime]",1986
2058,2141,"American Tail, An","[Adventure, Animation, Children, Comedy]",1986
2375,2459,"Texas Chainsaw Massacre, The",[Horror],1974
2404,2488,Peeping Tom,"[Drama, Horror, Thriller]",1960
2491,2575,"Dreamlife of Angels, The (Vie rêvée des anges,...",[Drama],1998
4407,4501,"Elvira, Mistress of the Dark","[Comedy, Horror]",1988
4408,4502,Ernest Saves Christmas,"[Children, Comedy]",1988
4582,4676,Troop Beverly Hills,[Comedy],1989
4927,5022,"Servant, The",[Drama],1963
5052,5147,Wild Strawberries (Smultronstället),[Drama],1957


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.


In [159]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
13159,63515,The Island,"[Drama, Mystery]",2006,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21419,103688,"Conjuring, The","[Horror, Thriller]",2013,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22929,109093,Don 2,"[Action, Crime, Mystery, Romance, Thriller]",2011,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23649,111856,7500,"[Action, Horror, Mystery, Thriller]",2014,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29312,134332,Bhool Bhulaiyaa,"[Comedy, Horror, Thriller]",2007,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.


In [161]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.


In [162]:
inputMovies['rating']

0    4.5000
1    3.1234
2    4.7321
3    4.8000
4    5.0000
Name: rating, dtype: float64

In [163]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure              0.0000
Animation              0.0000
Children               0.0000
Comedy                 5.0000
Fantasy                0.0000
Romance                4.7321
Drama                  4.5000
Action                 9.5321
Crime                  4.7321
Thriller              17.6555
Horror                12.9234
Mystery               14.0321
Sci-Fi                 0.0000
IMAX                   0.0000
Documentary            0.0000
War                    0.0000
Musical                0.0000
Western                0.0000
Film-Noir              0.0000
(no genres listed)     0.0000
dtype: float64

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.


Let's start by extracting the genre table from the original dataframe:


In [164]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [165]:
genreTable.shape

(34208, 20)

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.


In [166]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.068393
2    0.000000
3    0.133121
4    0.194674
5    0.068393
dtype: float64

In [167]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
81132    0.935272
27781    0.869615
76153    0.823227
75408    0.823227
8369     0.805326
dtype: float64

Now here's the Final recommendation table!


In [195]:
#num is the Number of recommendations according to your content from which you want to get top num_of_movies movies which users similar to you liked
num = 100
#num_of_movies is the Number of recommendations
num_of_movies = 20
Final = pd.DataFrame()
Final = movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(num).keys())]

y = []
mydefault = 0;
try:
    recommendation_df_collab['weighted average recommendation score'].loc[recommendationTable_df.index[0]]
except KeyError:
    x = mydefault
for i in range(0,num):
    x = 1
    try:
        recommendation_df_collab['weighted average recommendation score'].loc[recommendationTable_df.index[i]]
    except KeyError:
        x = mydefault
    if x==1:
        y.append(recommendation_df_collab['weighted average recommendation score'].loc[recommendationTable_df.index[i]])
    else:
        y.append(-1)
        
Final.insert(loc = 2,column = 'predicted rating',value = y)
Final_result = Final.sort_values(by='predicted rating',ascending=False)
Final_result.head(num_of_movies)



,movieId,title,predicted rating,genres,year
7988,8670,"Testament of Dr. Mabuse, The (Das Testament de...",5.000000,"[Crime, Horror, Mystery, Thriller]",1933
25703,119695,The Evictors,5.000000,"[Crime, Drama, Horror, Mystery, Thriller]",1979
3445,3535,American Psycho,4.500000,"[Crime, Horror, Mystery, Thriller]",2000
24519,115258,Charlie Chan in Honolulu,4.500000,"[Comedy, Crime, Horror, Mystery, Thriller]",1938
13552,67070,Army of One (Joshua Tree),4.500000,"[Action, Adventure, Crime, Drama, Mystery, Thr...",1993
23649,111856,7500,4.250000,"[Action, Horror, Mystery, Thriller]",2014
17063,86295,Scream 4,3.812500,"[Comedy, Horror, Mystery, Thriller]",2011
22929,109093,Don 2,3.750000,"[Action, Crime, Mystery, Romance, Thriller]",2011
27751,128634,Invisible Ghost,3.700000,"[Action, Horror, Mystery, Thriller]",1941
15066,76134,Sherlock Holmes in Pearl of Death (Pearl of De...,3.666667,"[Crime, Horror, Mystery, Thriller]",1944
